In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# The Chess Interface and Game Loop

### Dependencies

In [ ]:
import chess
import chess.svg
import chess.polyglot
import chess.gaviota
import time
import random
import re
from IPython.display import clear_output, SVG, display, HTML
from typing import Callable, Union
from pathlib import Path

import import_ipynb
import Util
from Globals import Globals
from ChessAlgorithm import ChessAlgorithm

#### chess.Board.push_legal
Pushes a move to a board's move stack only if it is a valid, legal move. If the move is illegal or invalid, a `ValueError` is raised and the board state remains unaffected.

###### __<u>Arguments</u>__
``move (chess.Move):``  
The move that should be pushed to the move stack.

In [ ]:
def push_legal(self, move: chess.Move):
    if move not in self.legal_moves:
        raise ValueError('Illegal move')
    self.push(move)

chess.Board.push_legal = push_legal
del push_legal

<a id= 'OpeningEnd'></a>
For the implementation of the `Game` class to make sense, we must first define opening books and endgame tablebases.

**Opening books** contain logical moves to make in a variety of positions in the early game. These are tried and tested moves that professional chess players often use in these positions. Reading a move from an opening book is very quick, so our implementation makes use of such an opening book. The opening book we use is *The Baron's Polyglot Opening Book* (see [Pijl, R. \[2018\]](Bibliography.ipynb#PR18)).

**Endgame tablebases** are similar in concept to opening books, but are applied during the endgame. Positions during endgames where both players have few pieces are stored in these tablebases. When reading such a position, one can retrieve an endgame score. If this score is positive, the current side to move has a guaranteed checkmate in that many moves. If this score is negative, the current side to move is guaranteed to be checkmated by the opponent in that many moves, assuming perfect play from the opponent. If this score is $0$, a draw is forced. Endgame tablebases allow search algorithms to find if moves lead to checkmates or draws, even if the amount of plies leading up to that event exceeds their search depth. The endgame tablebases we use are the *Gaviota endgame tablebases* (see [Chess Programming Wiki \[2020a\]](Bibliography.ipynb#CPW20a)).  
If the algorithm checks if the endgame tablebases can be used on each move, there is an unnecessary loss in performance. Therefore, it is important to only probe the endgame tablebases if the game is actually in the endgame. 

### Game Class
Lets the user play a game of chess against an AI using a virtual chess board UI. Alternatively, two AIs may play against each other. The game may use any algorithm for the AI(s) to make their moves.
<br><br>

#### Game.\_\_init\_\_
The constructor for the `Game` class. Gets called whenever a new `Game` instance is created.

###### __<u>Arguments</u>__
``algorithm_white (type[ChessAlgorithm], default: None):``  
The class of the chess algorithm that the white pieces should use to make a new move, or `None` if the white pieces are player-controlled.  

``algorithm_black (type[ChessAlgorithm], default: None):``  
The class of the chess algorithm that the black pieces should use to make a new move, or `None` if the black pieces are player-controlled. 

``search_depth_white (int, default: 4):``  
The search depth for the white AI's search algorithm. 

``search_depth_black (int, default: 4):``  
The search depth for the black AI's search algorithm.

``opening_book (str, default: 'Resources/baron30.bin'):``  
The path to the desired opening book the AIs will use, or `None` if no opening book is desired.

``endgame_tablebase_dir (str, default: 'Resources/Gaviota'):``  
The path to the directory of the desired endgame tablebases the AIs will use, or `None` if no endgame tablebases are desired.

``use_heuristic (bool, default: True):``  
Whether or not to use a heuristic to evaluate moves. This is only `False` in the context of chess problems, where only checkmates need to be considered.

``fen (str, default: None):``  
The FEN code representing the state that the chess board should start in, or `None` if it should start in the standard starting layout.  

###### <b><u>Side effects</u></b>
If neither `algorithm_white` nor `algorithm_black` is set to be an AI, the game will exit. At least one AI is required to play the game.


In [ ]:
class Game:
    def __init__(
        self,
        algorithm_white: type[ChessAlgorithm]=None,
        algorithm_black: type[ChessAlgorithm]=None,
        search_depth_white: int=4,
        search_depth_black: int=4,
        opening_book: str='Resources/baron30.bin',
        endgame_tablebase_dir: str='Resources/Gaviota',
        use_heuristic: bool=True,
        fen: str=None
    ):
        self.opening_book = opening_book
        self.endgame_tablebase_dir = endgame_tablebase_dir

        self.test_playthrough = []
        self.problem_playthroughs = {}
        self.wins = []

        self.board = chess.Board() if fen is None else chess.Board(fen=fen)
        self.board.ending = False
        self.board.check_endgame_tablebase = False

        if self.endgame_tablebase_dir is not None:
            self.endgame_tablebase = chess.gaviota.open_tablebase(
                    self.endgame_tablebase_dir)

        self.move_times = { chess.WHITE: [], chess.BLACK: [] }

        if algorithm_white is not None:
            algorithm_white = algorithm_white(self.board, search_depth_white,
                    self.endgame_tablebase, use_heuristic)
        if algorithm_black is not None:
            algorithm_black = algorithm_black(self.board, search_depth_black,
                    self.endgame_tablebase, use_heuristic)

        self.algorithm = {
            chess.WHITE: algorithm_white,
            chess.BLACK: algorithm_black
        }
        self.is_ai = self.get_ai_state()

        # The game requires that at least one player is an AI.
        # It is not a multiplayer chess game.
        if not self.is_ai[chess.WHITE] and not self.is_ai[chess.BLACK]:
            raise ValueError('At least one player must be an AI')

#### Game: Class Variables  

``BOARD_DISPLAY_SIZE (int):``  
The size of the displayed chess board in pixels.

``board (chess.Board):``   
The chess board instance that's used for the game.

``algorithm (dict<chess.Color, ChessAlgorithm>):``   
The algorithms the AIs, playing white and/or black, use to make a move.

``is_ai (dict<chess.Color, bool>):``   
Whether a color is AI-controlled (`True`) or player-controlled (`False`).

``move_times (dict<chess.Color, list>):``   
The time the AI(s) took to make their move.

``test_playthrough (list<tuple<int, chess.Move, bool>>):``  
The progress of the current game playthrough (evaluated move score, move and if the endgame tablebases produced the move).

``problem_playthroughs (dict<str, list>):``   
A test playthrough list (see above) for each test, useful when debugging multiple chess problem tests.

``wins (list<chess.Color>):``   
A list of colors that have won in the previous tests (`None` in case of a draw).

``opening_book (str):``  
A path to a Polyglot opening book, or `None` if no opening book is in use.

``endgame_tablebase_dir (str):``  
A path to a Gaviota endgame tablebase directory, or `None` if no endgame tablebases are in use.

``endgame_tablebase (chess.gaviota.PythonTablebase):``   
The endgame tablebase object, or `None` if no endgame tablebases are in use.

In [ ]:
# Constants
Game.BOARD_DISPLAY_SIZE = 500

# Board variables
Game.board = None

# AI and player declarations
Game.algorithm = None
Game.is_ai = None

# Testing declarations
Game.move_times = None
Game.test_playthrough = None
Game.problem_playthroughs = None
Game.wins = None

# Opening book and endgame tablebase declarations
Game.opening_book = None
Game.endgame_tablebase_dir = None
Game.endgame_tablebase = None

#### Game.play  

Starts a game of chess. Allows the AI(s) and optionally a human player to make moves, updating the chess board accordingly and displaying it visually. There are two distinct ways a game can progress:

``(AI, AI):``  
In this case, two AIs, specified by the user in the constructor of the class, play against each other. The AIs make moves based on the specified algorithms, depending on the current board state. The AIs take turns.
  
``(AI, Player) - (Player, AI):``    
In this case, an AI, specified by the user in the constructor of the class, and a human player compete against each other. The AI makes the best move found by the algorithm based on the current board state. On the other hand, the player can specify the move to make via input (if it's valid and legal). The player and AI take turns.

###### <b><u>Arguments</u></b>
``chess_problem (str, default: None):``  
The name of the chess problem which should be executed, or `None` if no chess problem is used.

``search_depth_auto (bool, default: False):``  
Whether or not the search depth should automatically be set to solve a chess problem. In games that don't involve a chess problem, this should be `False`.

``automation (bool, default: True):``  
Whether or not the game progresses without user input. This should always be `True` if more than one chess problem is executed.

``max_depth (int, default: None):``  
Only applicable to chess problems. Chess problems whose solutions require more plies than this value are skipped. If set to `None`, no chess problems are skipped.  


######  <u>__Side effects__</u> 
- The state of the board is changed.  
- The chess board is visually displayed.
- The times the AI(s) take to make a move are tracked and saved.
- All moves are tracked and saved.
- A "winner message" is displayed upon the end of the game.
- The game is saved in a text file. The name of this file is the current Unix timestamp.


In [ ]:
def play(
    self,
    chess_problem: str=None,
    search_depth_auto: bool=False,
    automation: bool=True,
    max_depth: int=None
) -> None:

    # Set up a chess problem if required
    use_heuristic = (chess_problem is None)
    if chess_problem is None:
        problem, iterations = None, None
    else:
        problem, iterations = self.set_up_chess_problem(chess_problem)
        if max_depth is not None and iterations > max_depth:
            return
        
    # Play the game
    self.display(display_board_score=False, automation=True)
    while not self.board.is_game_over():
        if self.is_ai[self.board.turn]:
            # AI makes a move
            time_stamp = time.time()

            # Make move
            made_opening_move, opening_move = self.make_opening_move()
            if made_opening_move:
                # Move from the opening library
                score, move, used_endgame, move_list = 'opening_library', opening_move, False, []
            else:
                color = self.board.turn
                
                # For chess problems, automatically set the search depth high enough
                # to solve the problem
                if search_depth_auto and iterations is not None:
                    self.algorithm[color].search_depth = iterations
                
                score, move, used_endgame, move_list = self.algorithm[color].make_move()
                self.test_playthrough.append((score, move.uci(), used_endgame))

            # Register time it took to make the move
            move_time = time.time() - time_stamp
            self.move_times[not self.board.turn].append(move_time)
        else:
            # Human player makes a move
            move = self.make_move_human()
            score, used_endgame, move_list, move_time = 'Human_move', 'Human_move', [], 'Human_move'

        self.board.check_and_set_ending()
        if self.endgame_tablebase is not None: self.board.update_check_endgame_tablebase()
        self.display(score, move, used_endgame, move_list, move_time, automation=automation)

    # Display message after the game has ended
    is_draw = self.board.outcome() is None or self.board.outcome().winner is None
    winner = '' if is_draw \
        else 'White' if self.board.outcome().winner == chess.WHITE \
        else 'Black'
    winner_message = 'It\'s a draw!' if is_draw else f'{winner} is the winner!'
    print(f'The game has ended. {winner_message}\n')
    print(self.board.outcome()) 
    
    # Register chess problem info
    if problem is not None and automation:
        self.set_problem_playthrough(problem)

    self.save_game()

Game.play = play
del play

#### Game.display
Displays the chess board graphically. Optionally, it also displays the last move, the score of that last move, the predicted move sequence, whether or not the endgame tablebases were used to find that move and how long the AI took to find that move. Optionally, it also displays the total board score for the color that just made a move, using the function `chess.Board.board_score` in [Util.ipynb](Util.ipynb#BoardScore).

###### __<u>Arguments</u>__

``score (Union[int, str], default: None): ``  
The score of the last move. Can also be a string displaying information if a score isn't applicable. If `None`, no score is displayed.  

``move (chess.Move, default: None): ``  
The move that was last made. If `None`, no move is displayed.  

``used_endgame (Union[bool, str], default: None): ``  
Whether or not the endgame tablebases were used to find the last move. Can also be a string displaying information if this statistic isn't applicable. If `None`, this statistic is not displayed. 

``move_list (list<chess.Move>, default: None): ``  
The predicted sequence of moves that the algorithm found. If `None`, no move list is displayed.    

``move_time (Union[float, str], default: None): ``  
The time, in seconds, that the AI took to find the last move. Can also be a string displaying information if a search time isn't applicable. If `None`, no time is displayed.  

``display_board_score (bool, default: True): ``  
Whether or not to display the board score for the last player that made a move.
   
``automation (bool, default: True): ``  
Whether or not the game progresses without user input.  

###### __<u>Side effects</u>__
The current visual output is overwritten.


In [ ]:
def display(
    self,
    score: Union[int, str]=None,
    move: chess.Move=None,
    used_endgame: Union[bool, str]=None,
    move_list: list=None,
    move_time: Union[float, str]=None,
    display_board_score: bool=True,
    automation: bool=True
) -> None:

    # Display board
    clear_output(wait=True)
    try:
        last_move = self.board.peek()
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE,
                lastmove=last_move, check=self.get_check_square()) 
    except IndexError:
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE)
    display(board_visual)

    # Display stats
    seconds_string = '' if isinstance(move_time, str) else 'seconds'
    if move_time is not None: print(f'Search time: {move_time} {seconds_string}')
    if move is not None: print(f'Move: {move.uci()}')
    if move_list is not None and len(move_list) > 0:
        move_sequence = ', '.join( move.uci() for move in move_list )
        print(f'Predicted move sequence: {move_sequence}')
    if score is not None: print(f'Move score: {score}')
    if used_endgame is not None: print(f'Used endgame tablebase: {used_endgame}')
    if display_board_score:
        color_string = 'white' if not self.board.turn == chess.WHITE else 'black'
        print(f'Board score for {color_string}: {self.board.board_score(not self.board.turn)}')

    if not automation:
        input('Press any button to continue')

Game.display = display
del display

#### Game.get_check_square
Checks if a piece is giving check, and if so, returns the square that piece is on. Due to limitations of the chess library, only one check square can be displayed. Therefore, if multiple pieces are giving check, only one such square is returned.

######  __<u>Returns _(chess.Square)_</u>__
The square which gives check, or `None` if there is no check.
    

In [ ]:
def get_check_square(self) -> chess.Square:
    if self.board.is_check():
        return self.board.checkers().pop()
    return None
            
Game.get_check_square = get_check_square
del get_check_square

#### Game.make_opening_move
Makes a random move from the opening book, if available. If not, no move is made.

###### __<u>Returns _(bool, chess.Move)_</u>__
- Whether or not an opening move has been made.
- The move that was made, or `None` if no move has been made.

In [ ]:
def make_opening_move(self) -> (bool, chess.Move):
    if self.opening_book is None:
        return False, None

    with chess.polyglot.open_reader(self.opening_book) as reader:
        possible_moves = [ entry.move for entry in reader.find_all(self.board) ]
        if len(possible_moves) > 0:
            move = random.choice(possible_moves)
            self.board.push(move)
            self.test_playthrough.append(('opening_library', move.uci(), False))
            return True, move

    return False, None

Game.make_opening_move = make_opening_move
del make_opening_move

#### Game.get_ai_state
Defines which players are AIs depending on the `self.algorithm` dictionary.

###### __<u>Returns _(dict<chess.Color, bool>)_</u>__
- The color of the player (white or black).
- Whether or not that player is an AI.

In [ ]:
def get_ai_state(self) -> dict:
    return {
        color: self.algorithm[color] is not None
            for color in ( chess.WHITE, chess.BLACK )
    }

Game.get_ai_state = get_ai_state
del get_ai_state

#### Game.make_move_human
If the player is human, this method enables the human player to make a move through the input console. The moves should contain the starting and ending square, e.g. `e2e4`. A fifth character is required in the case of a promotion to denote the piece to promote to, e.g. `e7e8q` for promoting the pawn to a queen.

###### __<u>Returns _(chess.Move)_</u>__
The move that was made.

In [ ]:
def make_move_human(self) -> chess.Move:
    move = None
    input_prompt = 'Please input your move: '
    while True:
        try:
            move = chess.Move.from_uci(input(input_prompt))
            self.board.push_legal(move)
            self.test_playthrough.append(('Human_move', move.uci(), 'Human_move'))
            return
        except ValueError:
            input_prompt = 'Illegal move, please try again: '

    return move
                
Game.make_move_human = make_move_human
del make_move_human

#### Game.save_game
Saves the move stack as well as all the scores and endgame tablebase usages to a file. The file is stored in the `SavedGames` directory. The name consists of the Unix timestamp at the time of the file's creation. 
This file can then later be loaded.

In [ ]:
def save_game(self) -> None:
    try:
        path = './SavedGames'
        Path(path).mkdir(parents=True, exist_ok=True)
        filename = str(time.time()).replace('.', '_') + '.txt'
        
        file_contents = ''
        with open(path + '/' + filename, 'w') as f:
            file_contents = 'move_white, move_black | score_white, score_black | ' \
                    + 'endgameUsed_white , endgameUsed_black ; \n'

            playthrough = { chess.WHITE: [], chess.BLACK: [] }
            for i, entry in enumerate(self.test_playthrough):
                color = chess.WHITE if i % 2 == 0 else chess.BLACK
                playthrough[color].append(entry)

            for i in range(len(playthrough[chess.WHITE])):
                score_white, move_white, used_endgame_white = playthrough[chess.WHITE][i]
                try:
                    score_black, move_black, used_endgame_black = playthrough[chess.BLACK][i]
                except IndexError:
                    score_black, move_black, used_endgame_black = None, None, None
                entry = f'{i}. {move_white} , {move_black} | {score_white} , {score_black} ' \
                        + f'| {used_endgame_white} , {used_endgame_black} ;\n'
                file_contents += entry

            file_contents += str(self.board.outcome())
            f.write(file_contents)
    except OSError:
        print('Could not write to the save file')
            
Game.save_game = save_game
del save_game

#### Game.load_game
A saved game is loaded. The user must input the path to the saved file using an input console. The game will be recreated by loading the move stack from a saved game. Optionally, the user can play back all the moves that were made during the game, and resume the game from the current position at any point.

###### __<u>Arguments</u>__
``playback (bool, default: True):``  
Whether or not to play back all moves of the game.

###### __<u>Side effects</u>__  
Execution is paused until a valid file name is given as user input.

In [ ]:
def load_game(self, playback: bool=True) -> None:
    while True:
        try:
            filename = input('Input the path to your save file:\n')
            if not filename.endswith('.txt'):
                filename += '.txt'

            self.board.reset()
            self.board.clear_stack()
            if playback:
                self.display()
                
            with open(filename) as f:
                lines = f.readlines()[1:-1]
                for line in lines:
                    items = [
                        item for item in line.split()
                            if item not in [',', '|', ';']
                            and not re.match('[0-9]+\.', item)
                    ]
                    if len(items) != 6:
                        raise OSError

                    move_white, move_black, score_white, score_black, \
                            used_endgame_white, used_endgame_black = items
                    items_white = (move_white, score_white, used_endgame_white)
                    items_black = (move_black, score_black, used_endgame_black)

                    for move, score, used_endgame in (items_white, items_black):
                        if move == 'None':
                            self.play()
                            return

                        if playback:
                            decision = input("Input 'P' to play from this state, input anything " \
                                    + 'else to continue')
                            if decision.lower() == 'p':
                                self.play()
                                return
                            self.test_playthrough.append((score, move, used_endgame))
                            self.board.push_legal(chess.Move.from_uci(move))
                            self.display()
                            print(f'Score: {score}, move: {move}, used endgame tablebase: {used_endgame}')
                        else:
                            self.test_playthrough.append((score, move, used_endgame))
                            self.board.push_legal(chess.Move.from_uci(move))

                        self.board.check_and_set_ending()
                        if self.endgame_tablebase is not None: self.board.update_check_endgame_tablebase()

            return
        except OSError:
            print(f'Ther file coult not be found or the file is corrupted - please try again')
    
Game.load_game = load_game
del load_game

#### Game.set_up_chess_problem
Sets the board state to the starting state of a chess problem, using a FEN code. If the given chess problem is invalid, it asks the user for input. Furthermore, it finds the minimum amount of moves to solve the chess problem, which can be used as a search depth to theoretically guarantee that a search algorithm can find the optimal solution.


###### __<u>Arguments</u>__
``chess_problem (str):``  
The abbreviated name of the chess problem (see `Globals.CHESS_PROBLEMS`).

###### <u>__Returns _(str, int)___</u>
- The abbreviated name of the problem that is played.
- The minimum amount of plies to solve the chess problem.

###### __<u>Side effects</u>__
If no chess problem is given as an argument, the user is prompted for input.

In [ ]:
def set_up_chess_problem(self, chess_problem: str) -> (str, int):
    # Clear the test_playthrough list - this dict may have contained moves from
    # the last playthrough
    self.test_playthrough.clear()

    if chess_problem not in Globals.CHESS_PROBLEMS:
        for p in Globals.CHESS_PROBLEMS:
            print(f' {p},')
        problem = input('Please choose a chess problem from the above. \n' \
                + 'You can find further information about the given problems ' \
                + 'in Globals.ipynb\n')
        while problem not in Globals.CHESS_PROBLEMS:
            clear_output()
            for p in Globals.CHESS_PROBLEMS:
                print(f' {p},')
            problem = input('Wrong input, please try again. Type one of the '
                    + 'above abbreviations to initialze the test')
    else:
        problem = chess_problem

    if problem is not None:
        self.board.set_fen(Globals.CHESS_PROBLEMS[problem])

    iterations = Globals.CHESS_PROBLEM_PLY_COUNT[problem] \
            if problem in Globals.CHESS_PROBLEM_PLY_COUNT \
            else None
    return problem, iterations

Game.set_up_chess_problem = set_up_chess_problem
del set_up_chess_problem

#### Game.set_problem_playthrough
Registers a playthrough of the current chess problem as defined in `self.test_playthrough`. This is added to the `self.problems_playthrough` dictionary so that the playthrough is saved even if another test is run. Furthermore, the winning color is appended to a list of winners.

###### __<u>Arguments</u>__
``problem (str):``  
The abbreviated name of the chess problem.

In [ ]:
def set_problem_playthrough(self, problem: str) -> None:
    self.problem_playthroughs[problem] = list(self.test_playthrough)
    winner = None
    if self.board.outcome() is None or self.board.outcome().winner is None:
        winner = None 
    elif self.board.outcome().winner == chess.WHITE:
        winner = 'white'
    else:
        winner = 'black'
            
    self.wins.append(winner)
    
Game.set_problem_playthrough = set_problem_playthrough
del set_problem_playthrough